In [1]:
# !pip install tensorflow

In [128]:
import torch
import pandas as pd
import numpy as np
import tensorflow as tf
from torch.utils.data import Dataset, DataLoader
import os
from pprint import pprint
import matplotlib
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [168]:
movielens_data_file_url = (
    "http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
)
movielens_dir = "ml-100k/"

ratings_file = movielens_dir + "ratings.csv"
tags_file = movielens_dir + "tags.csv"
movies_file = movielens_dir + "movies.csv"

df = pd.read_csv(ratings_file)
tags = pd.read_csv(tags_file)
movies = pd.read_csv(movies_file)

encoder = LabelEncoder()
movies["movieId"] = encoder.fit_transform(movies["movieId"])
df["movieId"] = encoder.transform(df["movieId"])


In [169]:
max_user_id = df.userId.max()+1
max_movie_id = df.movieId.max()+1

In [170]:
class movieDataset(Dataset):
    def __init__(self,df):
        self.data = df

    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.data[idx][0],self.data[idx][1]
    
training_data = movieDataset(df[["userId","movieId"]].values)
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

In [171]:
# movie_model = tf.keras.Sequential([
#   tf.keras.layers.StringLookup(
#       vocabulary=unique_movie_titles, mask_token=None),
#   tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
# ])

# user_model = tf.keras.Sequential([
#   tf.keras.layers.StringLookup(
#       vocabulary=unique_user_ids, mask_token=None),
#   # We add an additional embedding to account for unknown tokens.
#   tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
# ])

# class MovielensModel(tfrs.Model):

#   def __init__(self, user_model, movie_model):
#     super().__init__()
#     self.movie_model: tf.keras.Model = movie_model
#     self.user_model: tf.keras.Model = user_model
#     self.task: tf.keras.layers.Layer = task

#   def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
#     # We pick out the user features and pass them into the user model.
#     user_embeddings = self.user_model(features["user_id"])
#     # And pick out the movie features and pass them into the movie model,
#     # getting embeddings back.
#     positive_movie_embeddings = self.movie_model(features["movie_title"])

#     # The task computes the loss and the metrics.
#     return self.task(user_embeddings, positive_movie_embeddings)

In [172]:
query = np.random.rand(8,2)
candidate = np.random.rand(8,2)

In [173]:
class rankingTask():
    def __init__(self):
        self.loss = torch.nn.CrossEntropyLoss(reduction="sum")
        pass
    
    def __call__(self, query_embeddings, candidate_embeddings):
        scores = torch.matmul(
            query_embeddings, torch.transpose(candidate_embeddings,0,1)
        )
        
        num_queries, num_candidates = scores.shape

        labels = torch.range(0, num_queries-1, dtype=int)
    
    
        loss = self.loss(input=scores, target=labels)
        
        return loss

    

from torchnlp.encoders import LabelEncoder

class MovieModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.user_model = torch.nn.Embedding(max_user_id, 64)
        self.movie_model = torch.nn.Embedding(max_movie_id, 64)
        self.task = rankingTask()

    def forward(self, x):
        user, movie = x
        user = self.user_model(user)
        movie = self.movie_model(movie)
        
        loss = self.task(user,movie)
        return loss
        

task = rankingTask()
query_torch = torch.tensor(query)
candidate_torch = torch.tensor(candidate)
loss = task(query_torch, candidate_torch)
pprint(loss)
model = MovieModel()

tensor(16.6783, dtype=torch.float64)


/Users/jafen/miniconda3/envs/hm37/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  del sys.path[0]


In [174]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in tqdm(enumerate(train_dataloader, 0), total=len(train_dataloader)):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        loss = model(data)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

  0%|          | 0/1576 [00:00<?, ?it/s]

/Users/jafen/miniconda3/envs/hm37/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  del sys.path[0]


[1,   200] loss: 109.011
[1,   400] loss: 88.016
[1,   600] loss: 76.038
[1,   800] loss: 67.802
[1,  1000] loss: 61.583
[1,  1200] loss: 56.697
[1,  1400] loss: 53.215


  0%|          | 0/1576 [00:00<?, ?it/s]

[2,   200] loss: 46.085
[2,   400] loss: 44.886
[2,   600] loss: 43.015
[2,   800] loss: 42.019
[2,  1000] loss: 41.201
[2,  1200] loss: 39.981
[2,  1400] loss: 38.892


...

[8,   200] loss: 26.583
[8,   400] loss: 26.658
[8,   600] loss: 26.895
[8,   800] loss: 27.014
[8,  1000] loss: 26.861
[8,  1200] loss: 27.020
[8,  1400] loss: 27.028


  0%|          | 0/1576 [00:00<?, ?it/s]

[9,   200] loss: 26.013
[9,   400] loss: 26.295
[9,   600] loss: 26.368
[9,   800] loss: 26.521
[9,  1000] loss: 26.514
[9,  1200] loss: 26.576
[9,  1400] loss: 26.559


  0%|          | 0/1576 [00:00<?, ?it/s]

[10,   200] loss: 25.590
[10,   400] loss: 25.892
[10,   600] loss: 26.006
[10,   800] loss: 26.169
[10,  1000] loss: 26.251
[10,  1200] loss: 26.189
[10,  1400] loss: 26.190
Finished Training


# Bruteforce

In [176]:
class BruteForceLayer(torch.nn.Module):
    def __init__(self,
               query_model,
               k: int = 14):
        super().__init__()
        
        self.query_model = query_model
        self.k = k 
    
    def index(self, candidates, identifiers=None):
        self._candidates = candidates
        
    def _compute_score(self, queries, candidates):
        scores = torch.matmul(
            queries, torch.transpose(candidates,0,1)
        )
        return scores
    
    def forward(self, queries):
        
        if self.query_model is not None:
            queries = self.query_model(queries)
        
        scores = self._compute_score(queries, self._candidates)
        
        values, indices = torch.topk(scores, k=self.k)

        return values, indices
        
        
brute_layer = BruteForceLayer(model.user_model, k=14)
# brute_layer.index(training_data.map(lambda x: model.movie_model(torch.tensor(x[1]))))
unique_movies_mapped = torch.stack(list(map(lambda x: model.movie_model(torch.tensor(x)), df.movieId.unique())))

brute_layer.index(unique_movies_mapped)
brute_layer.query_model
brute_layer(torch.tensor([42]))

(tensor([[3.1378, 2.9226, 2.8977, 2.7952, 2.7771, 2.7641, 2.7489, 2.5737, 2.5294,
          2.5054, 2.3818, 2.3684, 2.3226, 2.3221]], grad_fn=<TopkBackward0>),
 tensor([[5311, 8870, 7446, 2899,  493, 8669, 8968, 9693, 3814, 4927, 6578,  594,
          6654, 4646]]))

In [177]:
len(df[df['movieId'].isin(out)].movieId.unique())

14

In [182]:
out = brute_layer(torch.tensor([42]))[1].tolist()[0]

movies[movies['movieId'].isin(out)]

,movieId,title,genres
493,493,Bhaji on the Beach (1993),Comedy|Drama
594,594,Twister (1996),Action|Adventure|Romance|Thriller
2899,2899,Bring It On (2000),Comedy
3814,3814,Lenny (1974),Drama
4646,4646,My Architect: A Son's Journey (2003),Documentary
4927,4927,Dawn of the Dead (1978),Action|Drama|Horror
5311,5311,Danny Deckchair (2003),Comedy|Romance
6578,6578,Mr. Woodcock (2007),Comedy
6654,6654,Dedication (2007),Comedy|Drama|Romance
7446,7446,In a Better World (Hævnen) (2010),Drama


In [63]:
input = tf.random.normal(shape=(3,4,5,6))
k = 2
values, indices  = tf.math.top_k(input, k=k)
values.shape.as_list()


values.shape == indices.shape == input.shape[:-1] + [k]

<tf.Tensor: id=181, shape=(3, 4, 5, 2), dtype=float32, numpy=
array([[[[ 0.9498922 ,  0.631757  ],
         [ 1.6435536 ,  0.8469825 ],
         [ 1.2498153 ,  0.9115617 ],
         [ 1.5148249 ,  0.6074722 ],
         [ 2.6113012 ,  1.1880776 ]],

        [[ 2.0017805 ,  1.4652351 ],
         [ 0.59968555, -0.4353429 ],
         [ 1.9111247 ,  1.0156393 ],
         [ 0.39036638,  0.17205158],
         [ 1.5485777 ,  0.77463126]],

        [[ 0.78752095,  0.2539187 ],
         [ 0.64380395,  0.35777348],
         [ 2.1960433 ,  0.33730984],
         [ 3.711439  ,  0.6046017 ],
         [ 0.34124258, -0.06920326]],

        [[ 0.46970934,  0.00956344],
         [ 1.3757367 ,  1.1440824 ],
         [-0.01413176, -0.49423015],
         [ 0.48665693, -0.01314055],
         [ 1.5572727 ,  1.164261  ]]],


       [[[ 0.85188305,  0.5459003 ],
         [ 1.5107961 ,  1.5052613 ],
         [ 1.6794207 ,  0.75349265],
         [ 1.8437959 ,  0.6825228 ],
         [ 1.2404474 ,  1.1558993 ]],

 

In [10]:
# query_tf = tf.convert_to_tensor(query)
# candidate_tf = tf.convert_to_tensor(candidate)


# scores = tf.linalg.matmul(
#         query_tf, candidate_tf, transpose_b=True)

# loss = tf.keras.losses.CategoricalCrossentropy(
#         from_logits=True, reduction=tf.keras.losses.Reduction.SUM)

# num_queries = tf.shape(scores)[0]
# num_candidates = tf.shape(scores)[1]

# labels = tf.eye(num_queries, num_candidates)

# scores.shape, labels.shape

# loss = loss(y_true=labels, y_pred=scores)
# loss
# import torch
# import tensorflow as tf

# query = np.array([[2.0,2,],[4,4]])

# query_torch = torch.tensor(query)
# eye_torch = torch.range(0,1, dtype=int)
# torch_loss = torch.nn.CrossEntropyLoss(reduction="sum")
# print(f"Loss with torch {torch_loss(query_torch, eye_torch)}")



# query_tf = tf.convert_to_tensor(query)
# eye_tf = tf.eye(2,2)
# tf_loss = tf.keras.losses.CategoricalCrossentropy(
#         from_logits=True, reduction=tf.keras.losses.Reduction.SUM)


# print(f"Loss with tf {tf_loss(y_pred=query_tf, y_true=eye_tf)}")




2022-03-19 12:32:27.290660: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-19 12:32:27.291344: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


<tf.Tensor: id=74, shape=(), dtype=float64, numpy=16.68231201171875>